In [1]:
import keras
from keras.layers import Dense , Flatten , MaxPooling2D , Conv2D , Input
from keras.models import Sequential,load_model , Model
from keras.losses import SparseCategoricalCrossentropy
from keras.activations import relu
from keras.optimizers import Adam
from keras.applications import VGG16
from keras.preprocessing.image import load_img , array_to_img , ImageDataGenerator

In [2]:


!pip install kaggle

In [3]:
from google.colab import files
upload = files.upload()

Saving kaggle.json to kaggle.json


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d balabaskar/tom-and-jerry-image-classification

 97% 420M/435M [00:03<00:00, 194MB/s]
100% 435M/435M [00:03<00:00, 143MB/s]


In [6]:
!unzip -q tom-and-jerry-image-classification

In [7]:
dataPath = '/content/tom_and_jerry'
img_height = 224
img_width = 224
dataGen = ImageDataGenerator(
    rescale=1/255.,
    validation_split = 0.2
)
TrainData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='training',
)
validationData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='validation',
)

Found 4383 images belonging to 1 classes.
Found 1095 images belonging to 1 classes.


In [8]:
baseModel = VGG16(include_top=False, weights='imagenet', input_shape=((img_height, img_width, 3)))

for layer in baseModel.layers:
  layer.trainable = False

input_layer = Input(shape=(224,224,3))
modelOuput = baseModel(input_layer) # Pass the input through the VGG16 base model
fOutput = Flatten()(modelOuput) # Flatten the output and add additional fully connected layers
dense_1 = Dense(512, activation='relu')(fOutput)
dense2 = Dense(256, activation='relu')(dense_1)
output_layer = Dense(5, activation='softmax')(dense2)


58889256/58889256 [==============================] - 0s 0us/step


In [9]:
model = Model(inputs=input_layer, outputs=output_layer)
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 5)                 1285      
                                                                 
Total params: 27692869 (105.64 MB)
Trainable params: 12978181

In [11]:
history = model.fit(
        TrainData ,
        epochs = 50,
        validation_data =validationData
)

Epoch 1/50
137/137 [==============================] - 42s 239ms/step - loss: 1007.8673 - accuracy: 0.2254 - val_loss: 2790.8179 - val_accuracy: 1.0000
Epoch 2/50
137/137 [==============================] - 25s 184ms/step - loss: 11747.2432 - accuracy: 0.1971 - val_loss: 21868.9492 - val_accuracy: 0.0000e+00
Epoch 3/50
137/137 [==============================] - 26s 187ms/step - loss: 43665.2305 - accuracy: 0.1971 - val_loss: 50193.0625 - val_accuracy: 1.0000
Epoch 4/50
137/137 [==============================] - 26s 191ms/step - loss: 93532.4531 - accuracy: 0.2044 - val_loss: 100470.7891 - val_accuracy: 0.0000e+00
Epoch 5/50
137/137 [==============================] - 26s 188ms/step - loss: 151586.3750 - accuracy: 0.2044 - val_loss: 186818.3125 - val_accuracy: 0.0000e+00
Epoch 6/50
137/137 [==============================] - 26s 191ms/step - loss: 204810.0000 - accuracy: 0.1962 - val_loss: 330385.6875 - val_accuracy: 0.0000e+00
Epoch 7/50
137/137 [==============================] - 26s 189ms

In [12]:
model.save("TomAndJerry.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
